# Character-level recurrent sequence-to-sequence model

**Author:** Deeepwin, [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2017/09/29<br>
**Last modified:** 2020/04/26<br>
**Description:** Character-level recurrent sequence-to-sequence model.

## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short German sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. German sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.


## Setup


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


## Download the data


In [2]:
!!curl -O http://www.manythings.org/anki/deu-eng.zip
!!unzip deu-eng.zip


['Archive:  deu-eng.zip',
 'replace deu.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

## Configuration


In [3]:
batch_size = 64         # Batch size for training.
epochs = 100            # Number of epochs to train for.
latent_dim = 256        # Latent dimensionality of the encoding space.
num_samples = 10000     # Number of samples to train on.
data_path = "deu.txt"   # Path to the data txt file on disk.


## Prepare the data


In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)      # generates a list with all characters that are in the english text
    for char in target_text:
        if char not in target_characters:   # generates a list with all characters that are in the German text
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)  
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 83
Max sequence length for inputs: 15
Max sequence length for outputs: 51


In [5]:
print("encoder_input_data.shape:", encoder_input_data.shape)
print("decoder_input_data.shape:", decoder_input_data.shape)
print("decoder_target_data.shape:", decoder_target_data.shape)

encoder_input_data.shape: (10000, 15, 70)
decoder_input_data.shape: (10000, 51, 83)
decoder_target_data.shape: (10000, 51, 83)


## Summarizing the Data

- The English dataset has max. word length of 15 and a total of 70 characters (encoder_input_data)
- The German dataset has a max. of 51 words and total of 83 characters (decoder_input_data, decoder_target_data)


## Build the model


In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))  # num_encoder_tokens=70

# LSTM expects input to be (batch, time, features)
encoder = keras.layers.LSTM(latent_dim, return_state=True)      # latent_dim=256, set return_state=True to output the cell states as well
                                                                # which we later use to initialize the decoder
encoder_outputs, state_h, state_c = encoder(encoder_inputs)     # (None, 256), (None, 256), (None, 256), encoder_inputs=(None, None, 70) 

# We discard `encoder_outputs` and only keep the states
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))  # num_decoder_tokens=83

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")    # output is probability for each german character that exists in the data
decoder_outputs = decoder_dense(decoder_outputs)                                # (None, None, 83) 

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Train the model


In [7]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/100


2022-11-19 13:50:01.318117: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


125/125 [==============================] - 4s 11ms/step - loss: 1.3189 - accuracy: 0.6891 - val_loss: 1.1976 - val_accuracy: 0.6649
Epoch 2/100
125/125 [==============================] - 1s 6ms/step - loss: 0.9099 - accuracy: 0.7572 - val_loss: 0.9277 - val_accuracy: 0.7451
Epoch 3/100
125/125 [==============================] - 1s 7ms/step - loss: 0.7205 - accuracy: 0.8018 - val_loss: 0.8189 - val_accuracy: 0.7669
Epoch 4/100
125/125 [==============================] - 1s 6ms/step - loss: 0.6330 - accuracy: 0.8205 - val_loss: 0.7404 - val_accuracy: 0.7907
Epoch 5/100
125/125 [==============================] - 1s 7ms/step - loss: 0.5766 - accuracy: 0.8346 - val_loss: 0.6857 - val_accuracy: 0.8031
Epoch 6/100
125/125 [==============================] - 1s 7ms/step - loss: 0.5378 - accuracy: 0.8450 - val_loss: 0.6553 - val_accuracy: 0.8129
Epoch 7/100
125/125 [==============================] - 1s 7ms/step - loss: 0.5063 - accuracy: 0.8538 - val_loss: 0.6312 - val_accuracy: 0.8203
Epoch 8/10

2022-11-19 13:51:25.762011: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


## Run inference (sampling)

**REMEMBER:** Why do we need to decode iteratively? 

Because the German word can have a different length than the English when translating. The encoder encodes the context and temporal dependencies of the sequence into the hidden states. The hidden states represent the meaning of the input. The decoder must now translate that meaning into a German sentence. Each sentence has a different meaning. Therefore the decoder will be presented for each word by different meanings (hidden state). The decoder learns to associate these meanings with a German word.

**QUESTION:** Why can I not simply generate the German translation directly from the hidden states, without iteratively feed the decoder output back?

This is not clear to me. In the end you have to define anyway a max sequence length on the decoder. For shorter words with empty character at the end, NN can simply learn to set character probability to zero. 

With encoder-decoder model, during inference, it is completely inefficient to feedback the first prediction which is a vector of words [<start> pred_1 _ _ _ _] with all the empty words denoted as _ into the LSTM. The LSTM itself, will step through all these empty wordsto make the next prediction. 

During training, this is ok. Because we feed the y_true as single vector to LSTM and it will step through at once (internally iterating). That is efficient.

Maybe this is the answer: Because we train LSTM efficiently, in one shot, we cannot change regime or modify the model, but must infer in a similar way. That means we have to feed decoder something. Assuming the training was very successful, it makes correct predictions, we simply must get the next word, which can be used as input for the next iteration. E voila. 

The LSTM must only be able to predict the next word correctly! For that it just needs all inputs before current time step and not the empty words. All the remaining empty words _ and the output the LSTM will generate for them do not matter. We can simply ignore.

And this we do. When we take argmax() we only look at current word and prediction probability.

Best explanation I found so far is here: https://medium.com/analytics-vidhya/encoder-decoder-seq2seq-models-clearly-explained-c34186fbf49b 

In a summary:

- The original idea of se2seq model is using LSTM (recurrent networks)
- To make training efficient, teacher forcing is used, which dictates how decoder is defined
- Because we have no ground truth during inference, we must think about another way to feed the decoder input (a trick or tweek)
- We know, if decoder was trained successful, we can enter the start token and will get the next predicted word based on the hidden state
- We only need to predict the next work correctly, not the full sentence
- We can then feed back the correct prediction to the input to get the next correct prediction an build up the output sequence step by step


1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [10]:
# Define sampling models, restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):

    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

You can now generate decoded sentences as such:


In [12]:
for seq_index in range(20):
    
    # Take one sequence (part of the training set) for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: Go.
Decoded sentence: Geh.

-
Input sentence: Hi.
Decoded sentence: Grüß Gott!

-
Input sentence: Hi.
Decoded sentence: Grüß Gott!

-
Input sentence: Run!
Decoded sentence: Lauf!

-
Input sentence: Run.
Decoded sentence: Lauf!

-
Input sentence: Wow!
Decoded sentence: Potzdonner!

-
Input sentence: Wow!
Decoded sentence: Potzdonner!

-
Input sentence: Duck!
Decoded sentence: Kopf runter!

-
Input sentence: Fire!
Decoded sentence: Feuer!

-
Input sentence: Help!
Decoded sentence: Hilfe!

-
Input sentence: Help!
Decoded sentence: Hilfe!

-
Input sentence: Stay.
Decoded sentence: Bleib!

-
Input sentence: Stop!
Decoded sentence: Stopp!

-
Input sentence: Stop!
Decoded sentence: Stopp!

-
Input sentence: Wait!
Decoded sentence: Warte!

-
Input sentence: Wait.
Decoded sentence: Warte.

-
Input sentence: Begin.
Decoded sentence: Fang an.

-
Input sentence: Do it.
Decoded sentence: Mache es!

-
Input sentence: Do it.
Decoded sentence: Mache es!

-
Input sentence: Go on.
Deco